In [4]:
# install necessary libraries
#!pip install scikit-learn
#!pip install pandas
#!pip install nltk

#download necessary nltk resources
import nltk
#nltk.download('punkt')
#nltk.download('punkt_tab')
#nltk.download('wordnet')
#nltk.download('stopwords')

#print("NLTK Resources downloaded successfully!")

# Import necessary libraries
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
# 🌟 替换：导入 CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Import Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from joblib import dump,load

# Import NLTK
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

# Initialize NLTK resources
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Placeholder: Load the actual dataset. Ensure it has 'text' (user query) and 'intent' (label) columns
df = pd.read_csv('dataset.csv')


#shuffle the data for robust splitting
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df.head())

def preprocess_text(text):
    # 1. Convert to Lowercase
    text = text.lower()
    
    # 2. Remove Punctuation and Special Characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # 3. Tokenization
    tokens = word_tokenize(text)
    
    # 4. Stopword Removal
    tokens = [word for word in tokens if word not in stop_words]
    
    # 5. Lemmatization (Key Enhancement)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Rejoin tokens into a single string
    return ' '.join(tokens)

# Apply the new preprocessing function to the text column
df['cleaned_text'] = df['text'].apply(preprocess_text)
print("--- Preprocessing Complete (with NLTK Lemmatization) ---")
print(df[['text', 'cleaned_text']].head())

# Prepare the cleaned text and intents for the model training section
X = df['cleaned_text']
y = df['intent']

# 🌟 替换：初始化 CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the text data to create the feature matrix X
X = vectorizer.fit_transform(X)
y = df['intent']

print(f"Feature matrix X shape: {X.shape}")
print(f"Labels y shape: {y.shape}")


X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    test_size=0.2,
    random_state=42,
    # 仅在类别数大于1时使用 stratify
    stratify=y if len(df['intent'].unique()) > 1 else None, 
)

print(f"Train set size:{X_train.shape[0]} samples")
print(f"Test set size:{X_test.shape[0]} samples")

# Instantiate the Multiomial Naive Bayes model (通常与词频 Count 特征配合得很好)
nb_model = MultinomialNB()

# Train the model
nb_model.fit(X_train, y_train)

# Make predictions on the test set
pred = nb_model.predict(X_test)

print("--- Naive Bayes Model Evaluation (using CountVectorizer) ---")
print("Accuracy:", accuracy_score(y_test, pred))
print("\nClassification Report:\n")
if len(df['intent'].unique()) > 1:
    print(classification_report(y_test, pred, zero_division=0))
else:
    print("Classification Report skipped: Only one class in test set.")

# Save the trained model and the vectorizer (essential for transforming new user input)
# ⚠️ 注意：我们将 vectorizer 的文件名更改为反映 CountVectorizer
#dump(nb_model, 'naive_bayes_intent_model.joblib')
#dump(vectorizer, 'count_vectorizer.joblib')
#print("Model and Vectorizer saved using joblib.")

# Predefined fixed responses (Retrieval System)
responses = {
    "ask_room_price": "Our rooms start from RM180 per night.",
    "ask_availability": "We currently have several rooms available.",
    "ask_facilities": "We offer free Wi-Fi, breakfast, pool, gym and parking.",
    "ask_location": "We are located in Kuala Lumpur City Centre (KLCC).",
    "ask_checkin_time" : "Check-in time is from 2:00 PM.",
    "ask_checkout_time" : "Check-out time is at 12:00 PM.",
    "ask_booking" : "You can book directly through our website or at the front desk.",
    "ask_cancellation" : "Cancellations are free up to 24 hours before arrival.",
    "greeting" : "Hello! How may I assist you today?",
    "goodbye" : "Goodbye! Have a great day!"
}

def chatbot_reply_nb(user_input, model, vectorizer, responses):
    # 1. Preprocessing
    # ⚠️ 关键：对输入文本进行完整预处理，与训练阶段保持一致
    user_input_cleaned = preprocess_text(user_input)

    # 2. Feature Extraction: Transform the input using the fitted vectorizer
    vector = vectorizer.transform([user_input_cleaned])

    # 3. Intent Prediction
    intent = model.predict(vector)[0]

    # 4. Retrieval (Check for unknown intent/fallback)
    # If the predicted intent exists in the dictionary, return the specific response
    # Otherwise, return a fallback message
    return responses.get(intent, f"Sorry, I predicted the intent '{intent}', but I don't have a specific response for that yet. Please rephrase your question.")

# Test the chatbot function
print("\n --- Naive Bayes Chatbot Test (CountVectorizer) ---")
test_input = "When should I check out?"
predicted_response = chatbot_reply_nb(test_input, nb_model, vectorizer, responses)
print(f"User Input: {test_input}")
print(f"Chatbot Reply: {predicted_response}")

                                text             intent
0                       Good evening           greeting
1           When does check-out end?  ask_checkout_time
2  What is your cancellation policy?   ask_cancellation
3      What is your check-in policy?   ask_checkin_time
4          When does check-in start?   ask_checkin_time
--- Preprocessing Complete (with NLTK Lemmatization) ---
                                text         cleaned_text
0                       Good evening         good evening
1           When does check-out end?         checkout end
2  What is your cancellation policy?  cancellation policy
3      What is your check-in policy?       checkin policy
4          When does check-in start?        checkin start
Feature matrix X shape: (100, 105)
Labels y shape: (100,)
Train set size:80 samples
Test set size:20 samples
--- Naive Bayes Model Evaluation (using CountVectorizer) ---
Accuracy: 0.75

Classification Report:

                   precision    recall  f1-score 